# Worksheet 08

Name:  Sumatra Dhimoyee

UID:   U26967910

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.207276643999312, 5.040991006678501, 5.7106852050157375, 3.8788999930644774, 6.542422255326369, 5.612540824820744, 6.5160486667048065, 5.334576566259185, 5.296687386916833, 4.598331535860882]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.561793244174449, 8.30792532728547, 6.585415806692017, 7.077238394141636, 8.813859460861815, 9.598073450595814, 8.79262305770793, 7.942267841334683, 6.792070229996007, 7.4244576929823545]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.4244576929823545, 6.792070229996007, 4.598331535860882, 7.942267841334683, 8.79262305770793, 5.296687386916833, 9.598073450595814, 8.813859460861815, 7.077238394141636, 5.334576566259185]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The proability of the species, the mean and variance of the species




d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.4244576929823545, 7.942267841334683, 8.79262305770793, 9.598073450595814, 8.813859460861815, 7.077238394141636]
[6.792070229996007, 4.598331535860882, 5.296687386916833, 5.334576566259185]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.274753316270706,  mean_2 = 5.505416429758227
var_1 = 0.7629319848151536,  var_2 = 0.6377587696722338


The values produced are relativitly close to there true value of the values inputted

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]
    
    #pdf_i[0] is pr(p|s_0)
    
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    
    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0]* prob_s[0]/prob_x))
    prob_s1_x.append( (pdf_i[1]* prob_s[1]/prob_x) )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.4244576929823545
probability of observing that point if it came from cluster 0 =  0.2809948763338073
probability of observing that point if it came from cluster 1 =  0.006762907309000958
point =  6.792070229996007
probability of observing that point if it came from cluster 0 =  0.07912260736458315
probability of observing that point if it came from cluster 1 =  0.08173941739495506
point =  4.598331535860882
probability of observing that point if it came from cluster 0 =  4.743187052818045e-06
probability of observing that point if it came from cluster 1 =  0.22749811284433977
point =  7.942267841334683
probability of observing that point if it came from cluster 0 =  0.4755359489230114
probability of observing that point if it came from cluster 1 =  0.0004226293538147006
point =  8.79262305770793
probability of observing that point if it came from cluster 0 =  0.4153103087788675
probability of observing that point if it came from cluster 1 =  1.0649305638253262e-06
point =  5

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x), ] #Look at GMM Clustering uj equation
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.161793784343129,  mean_2 = 5.350660610520558
var_1 = 0.8716618594245422,  var_2 = 0.5182759389790692


For all values, there is noticable changes when compared to before in which all the values output above have increased in value.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]
    
    #pdf_i[0] is pr(p|s_0)
    
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    
    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0]* prob_s[0]/prob_x))
    prob_s1_x.append( (pdf_i[1]* prob_s[1]/prob_x) )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.4244576929823545
probability of observing that point if it came from cluster 0 =  0.32002516822090593
probability of observing that point if it came from cluster 1 =  0.0002568436631859319
point =  6.792070229996007
probability of observing that point if it came from cluster 0 =  0.13315742741220077
probability of observing that point if it came from cluster 1 =  0.016096980798222724
point =  4.598331535860882
probability of observing that point if it came from cluster 0 =  0.00010750577369726864
probability of observing that point if it came from cluster 1 =  0.26840411142279175
point =  7.942267841334683
probability of observing that point if it came from cluster 0 =  0.4433931491644272
probability of observing that point if it came from cluster 1 =  2.862296523628604e-06
point =  8.79262305770793
probability of observing that point if it came from cluster 0 =  0.35223314808577477
probability of observing that point if it came from cluster 1 =  2.0371287048446265e-10
point

The majority of probabilities also increased for S_1 while the probability coming from S_2 decreased (although negliable)

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:

hard_assignments = [] 

for p, prob_s0, prob_s1 in zip(data, prob_s0_x, prob_s1_x):
    if prob_s0 > prob_s1:
        hard_assignments.append(0)
    else:
        hard_assignments.append(1)

for i, assignment in enumerate(hard_assignments):
    print(f"Point {data[i]} belongs to cluster {assignment}")


Point 7.4244576929823545 belongs to cluster 0
Point 6.792070229996007 belongs to cluster 0
Point 4.598331535860882 belongs to cluster 1
Point 7.942267841334683 belongs to cluster 0
Point 8.79262305770793 belongs to cluster 0
Point 5.296687386916833 belongs to cluster 1
Point 9.598073450595814 belongs to cluster 0
Point 8.813859460861815 belongs to cluster 0
Point 7.077238394141636 belongs to cluster 0
Point 5.334576566259185 belongs to cluster 1
